1. You should try to build the best, stacking-based model(s) to predict age.

2. The UC Irvine Machine Learning Repository classifies this dataset as a "classification" dataset, but age is stored as a numeric (albeit discrete-valued) variable. So, I think it could maybe be reasonable to treat this as a regression problem. It's up to you!

In [1]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\achur\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [14]:

#from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
  
# fetch dataset 
#abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
#X = abalone.data.features 
#y = abalone.data.targets 
  
# metadata 
#print(abalone.metadata) 
  
# variable information 
#print(abalone.variables) 

#X = pd.get_dummies(X)
column_names = [
    'Sex', 'Length', 'Diameter', 'Height',
    'WholeWeight', 'ShuckedWeight',
    'VisceraWeight', 'ShellWeight', 'Rings'
]
abalone = pd.read_csv("abalone.data", header=None, names=column_names)

X = pd.get_dummies(abalone.drop("Rings", axis =1), drop_first=True)
y = abalone["Rings"]


# Stacking Model:

In [10]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.Collecting matplotlib
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------------------------ --- 7.3/8.1 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 23.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------  2.6/2.6 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 5.6 MB/s eta 0:00:00



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\achur\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [15]:
import sklearn

from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot
 
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    level0.append(('svm', SVR()))
    # define meta learner model
    level1 = LinearRegression()
    # define the stacking ensemble
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    return model


# get a list of models to evaluate
def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['svm'] = SVR()
    models['stacking'] = get_stacking()
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# define x and y
X = pd.get_dummies(abalone.drop("Rings", axis=1), drop_first=True)
y = abalone["Rings"]

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


>knn 0.514 (0.031)
>cart 0.087 (0.113)
>svm 0.515 (0.034)
>stacking 0.559 (0.031)


In [17]:
# make predictions
stacking_model = get_stacking()
stacking_model.fit(X, y)

y_pred = stacking_model.predict(X)
avg_y_pred = mean(y_pred)
print(avg_y_pred)

9.990195815531669


The average age for abalone is 9.99.